In [1]:
import os
import pandas as pd

from ultralytics import YOLO

In [2]:
LABELS = {
    0: "aegypti",
    1: "albopictus",
    2: "anopheles",
    3: "culex",
    4: "culiseta",
    5: "japonicus/koreicus"
}

# Bounding Box Format: [x_center, y_center, width, height]
# Output Format: [id, ImageID, LabelName, Conf, xcenter, ycenter, bbx_width, bbx_height] (Indexed from 0)

In [3]:
# Loading Model
model = YOLO(model='yolov8m.pt', task='detect', verbose=True)

# Loading Model in GPU
model = model.cuda()

arguments = {
    "project": "yolo-experiments",
    "name": "yolov8m-3",
    "data": "./data.yaml",
    "imgsz": 640,
    "batch": -1,
    "epochs": 20,
    "patience": 3,
    "seed": 69,
    
    "box": 8.0,
    "cls": 2.0,
    "dfl": 6.0,
    "pose": 6.0,
    "kobj": 1.0,
    "plots": True,
    
    "hsv_v": 1.0,
    
}

In [4]:
# Model Training
results = model.train(**arguments)

engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=./data.yaml, epochs=20, time=None, patience=3, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=yolo-experiments, name=yolov8m-3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=69, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=Fa

train: Scanning /home/bersi/Documents/Academic_Stuff/MCV-DataContest/data/labels/train.cache... 6750 images, 0 backgrounds, 1 corrupt: 100%|██████████| 6750/6750 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/bersi/Documents/Academic_Stuff/MCV-DataContest/data/images/train/120b30b0-c7db-4f0a-bead-a30424a65453.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0068]



val: Scanning /home/bersi/Documents/Academic_Stuff/MCV-DataContest/data/labels/val.cache... 750 images, 0 backgrounds, 0 corrupt: 100%|██████████| 750/750 [00:00<?, ?it/s]


Plotting labels to yolo-experiments/yolov8m-3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.000515625), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo-experiments/yolov8m-3
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        750        750      0.572      0.389      0.355      0.259

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      13/20       3.2G      1.177      4.011      5.453          5        640: 100%|██████████| 1125/1125 [03:57<00:00,  4.74it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:09<00:00,  6.45it/s]


                   all        750        750      0.573      0.422      0.355      0.263
EarlyStopping: Training stopped early as no improvement observed in last 3 epochs. Best results observed at epoch 10, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

13 epochs completed in 0.875 hours.
Optimizer stripped from yolo-experiments/yolov8m-3/weights/last.pt, 52.0MB
Optimizer stripped from yolo-experiments/yolov8m-3/weights/best.pt, 52.0MB

Validating yolo-experiments/yolov8m-3/weights/best.pt...
Ultralytics YOLOv8.2.1 🚀 Python-3.12.3 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
Model summary (fused): 218 layers, 25843234 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:08<00:00,  7.81it/s]


                   all        750        750      0.767      0.375      0.385      0.274
               aegypti        750          2          1          0      0.248      0.151
            albopictus        750        328      0.772      0.909      0.911      0.673
             anopheles        750          6          1          0     0.0258     0.0208
                 culex        750        340      0.634      0.915      0.833      0.572
              culiseta        750         49      0.197      0.429       0.21      0.168
    japonicus/koreicus        750         25          1          0     0.0828     0.0623
Speed: 0.2ms preprocess, 6.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to yolo-experiments/yolov8m-3


In [5]:
import os

TRAIN_DIR = "../data/images/train"
VAL_DIR = "../data/images/val"
TEST_DIR = "../data/images/test"

train_images = os.listdir(TRAIN_DIR)
val_images = os.listdir(VAL_DIR)
test_images = os.listdir(TEST_DIR)

In [6]:
results_test_dataframe = pd.DataFrame(columns=["id", "ImageID", "LabelName", "Conf", "xcenter", "ycenter", "bbx_width", "bbx_height"])

# model = YOLO("./yolo-experiments/yolov8m-2/weights/best.pt", task="detect", verbose=True)

results = model(TEST_DIR, stream=True, conf=0)

In [7]:
for id, r in enumerate(results):
    r = r.cpu()
    img_path = r.path
    img_name = img_path.split("/")[-1]
    boxes = r.boxes.numpy()
    label = boxes.cls
    conf = boxes.conf
    xywh = boxes.xywhn
    
    new_row = {"id": id, "ImageID": img_name, "LabelName": LABELS[int(label[0])], "Conf": conf[0], "xcenter": xywh[:, 0][0], "ycenter": xywh[:, 1][0], "bbx_width": xywh[:, 2][0], "bbx_height": xywh[:, 3][0]}
    results_test_dataframe = pd.concat([results_test_dataframe, pd.DataFrame([new_row])], ignore_index=True, copy=False)


image 1/525 /home/bersi/Documents/Academic_Stuff/MCV-DataContest/notebooks/../data/images/test/0031063e-716a-4080-934c-77598dc8de72.jpeg: 640x480 126 albopictuss, 172 culexs, 1 culiseta, 1 japonicus/koreicus, 70.8ms
image 2/525 /home/bersi/Documents/Academic_Stuff/MCV-DataContest/notebooks/../data/images/test/00fbfad7-9722-4581-831c-79faa576ea7f.jpeg: 640x480 119 albopictuss, 179 culexs, 2 japonicus/koreicuss, 13.7ms
image 3/525 /home/bersi/Documents/Academic_Stuff/MCV-DataContest/notebooks/../data/images/test/02043b0e-3d7d-4ca4-a36f-4bf97c344264.jpeg: 640x384 75 albopictuss, 224 culexs, 1 culiseta, 83.4ms
image 4/525 /home/bersi/Documents/Academic_Stuff/MCV-DataContest/notebooks/../data/images/test/0365513c-8f00-44f3-abd0-1fadde81c602.jpeg: 640x480 71 albopictuss, 228 culexs, 1 culiseta, 14.5ms
image 5/525 /home/bersi/Documents/Academic_Stuff/MCV-DataContest/notebooks/../data/images/test/0365d78b-2064-4d54-b15c-994d1950479a.jpeg: 640x512 68 albopictuss, 231 culexs, 1 japonicus/koreic

In [8]:
results_test_dataframe.to_csv("./submissions/results_test_3.csv", index=False)